In [11]:
import time
import pickle
import numpy as np
import pandas as pd

In [12]:
text_file = r"E:\Python_Workplace\OptimalDistinct\Cluster\data\sbert_similarity_results_allcat_with_language.csv"
df = pd.read_csv(text_file)
df.columns

Index(['GPTs_ID', 'GPTs_Name', 'Description', 'Rating', 'Number of Ratings',
       'Conversions', 'Author URL', 'Author', 'Features', 'Conversion Start',
       'num_rate', 'Website', 'Linkedin', 'Github', 'Twitter', 'num_of_links',
       'map_Category', 'gpts_description_processed_feature',
       'gpts_description_processed_trending',
       'gpts_description_processed_allcat', 'text_combine', 'top10',
       'Features_processed', 'Conversion Start_processed',
       'Description_processed', 'text_combine_processed',
       'gpts_description_processed_top10', 'similarity_desc_feature',
       'similarity_desc_trending', 'similarity_desc_allcat',
       'similarity_desc_top10', 'Primary_Language'],
      dtype='object')

In [15]:
df["Conversion_Start"]

KeyError: 'Conversion_Start'

In [5]:
text_file = r".\data\Detail_99_CleanData_0507_with_language.csv"
df = pd.read_csv(text_file)
# df.columns
language_summary = df['Primary_Language'].value_counts()
print(language_summary)

en       8261
zh-cn     376
ja        348
ko        226
fr        197
es        123
pt         95
de         83
it         65
ca         61
ro         35
nl         31
no         28
da         25
vi         21
ru         16
af         15
None       14
pl         14
cs         11
id         10
sv         10
ar         10
tl          8
et          6
zh-tw       6
fi          6
sl          3
sk          3
bg          3
tr          3
cy          2
sw          2
el          2
lv          1
hr          1
uk          1
th          1
so          1
hu          1
Name: Primary_Language, dtype: int64
